In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import numpy as np

In [2]:
# Load and concatenate the CSV files from 2000 to 2024
years = list(range(2000, 2025))
dfs = []

# Assuming all the CSV files are in the format 'daily_aqi_by_county_<year>.csv'
for year in years:
    file_name = f"daily_aqi_data/daily_aqi_by_county_{year}.csv"
    df = pd.read_csv(file_name)
    dfs.append(df)

print(df.head())

/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_25276/2848292098.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_25276/2848292098.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_25276/2848292098.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_25276/2848292098.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
/var/folders/n8/089mkp9j15jc90lqvrpw8yym0000gn/T/ipykernel_25276/2848292098.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on im

  State Name county Name  State Code  County Code        Date  AQI Category  \
0    Alabama     Baldwin           1            3  2024-01-03   41     Good   
1    Alabama     Baldwin           1            3  2024-01-04   38     Good   
2    Alabama     Baldwin           1            3  2024-01-05   44     Good   
3    Alabama     Baldwin           1            3  2024-01-06    7     Good   
4    Alabama     Baldwin           1            3  2024-01-07   29     Good   

  Defining Parameter Defining Site  Number of Sites Reporting  
0              PM2.5   01-003-0010                          1  
1              PM2.5   01-003-0010                          1  
2              PM2.5   01-003-0010                          1  
3              PM2.5   01-003-0010                          1  
4              PM2.5   01-003-0010                          1  


In [3]:
# Concatenate all years' data into one DataFrame
data = pd.concat(dfs, ignore_index=True)

# Convert 'Date' column to datetime format and extract 'Year'
data["Date"] = pd.to_datetime(data["Date"])
data["Year"] = data["Date"].dt.year

# Group by 'State Name' and 'Year' to get the average AQI for each state per year
state_yearly_aqi = data.groupby(["State Name", "Year"])["AQI"].mean().reset_index()

In [4]:
# ----- Chart 1: Interactive Line Chart for AQI Trends by State (2000–2024) -----

# Create an interactive line chart for AQI trends by state over time
line_chart = (
    alt.Chart(state_yearly_aqi)
    .mark_line()
    .encode(
        x=alt.X("Year:O", title="Year"),
        y=alt.Y("AQI:Q", title="Average AQI"),
        color="State Name:N",  # Color by state
        tooltip=["State Name", "Year", "AQI"],  # Tooltip showing state, year, and AQI
    )
    .properties(title="Annual Average AQI by State (2000–2024)", width=700, height=400)
    .interactive()
)  # Make the chart interactive (zoom, pan)

In [5]:
# ----- Chart 2: Interactive Bar Chart for State AQI Comparison (2024) -----

# Filter data for a specific year (e.g., 2024)
data_2024 = state_yearly_aqi[state_yearly_aqi["Year"] == 2024]

# Create an interactive bar chart for AQI by state for a specific year (2024)
bar_chart = (
    alt.Chart(data_2024)
    .mark_bar()
    .encode(
        x=alt.X("State Name:N", sort="-y", title="State"),
        y=alt.Y("AQI:Q", title="Average AQI"),
        color="AQI:Q",  # Color by AQI values
        tooltip=["State Name", "AQI"],  # Tooltip showing state and AQI
    )
    .properties(title="Average AQI by State in 2024", width=700, height=400)
    .interactive()
)  # Make the chart interactive

# ----- Combine Both Charts -----

# Combine the two charts vertically (one on top of the other)
combined_chart = line_chart & bar_chart

# Display the combined chart
combined_chart

alt.VConcatChart(...)

In [ ]:
state_yearly_aqi = state_yearly_aqi[
    state_yearly_aqi["State Name"] != "Country Of Mexico"
]

# Define the year parameter with a slider selection
year_param = alt.param(
    name="Select_Year",  # Signal name
    bind=alt.binding_range(
        min=2000,  # Minimum year
        max=2024,  # Maximum year
        step=1,  # Step for the slider
        name="Year: ",
    ),
    value=2024,  # Default value
)

# Create an interactive bar chart for AQI by state for a specific year
bar_chart = (
    alt.Chart(state_yearly_aqi)
    .mark_bar()
    .encode(
        x=alt.X("State Name:N", sort="-y", title="State"),
        y=alt.Y("AQI:Q", title="Average AQI"),
        color=alt.Color("AQI:Q", title="Average AQI"),
        tooltip=["State Name", "AQI", "Year"],
    )
    .add_params(year_param)  # Add the year parameter to the chart
    .transform_filter(alt.datum.Year == year_param)  # Filter based on selected year
    .properties(title="Average AQI by State for Selected Year", width=700, height=400)
)

bar_chart

alt.Chart(...)

In [7]:
# ----- Interactive State Selector -----

# Create a selection parameter for the state
state_param = alt.param(
    "Select_State",  # Signal name (use underscores instead of spaces)
    bind=alt.binding_select(
        options=state_yearly_aqi["State Name"].unique(), name="Select State: "
    ),
    value="California",  # Default value
)

# Line chart for the selected state with tooltips
line_chart = (
    alt.Chart(state_yearly_aqi)
    .mark_line(point=True)
    .encode(
        x=alt.X("Year:O", title="Year"),
        y=alt.Y(
            "AQI:Q",
            title="Average AQI",
            scale=alt.Scale(domain=[0, state_yearly_aqi["AQI"].max() + 10]),
        ),
        color=alt.value("#1f77b4"),  # Professional blue color for the line
        tooltip=["State Name", "Year", "AQI"],  # Tooltip showing state, year, and AQI
    )
    .transform_filter(
        alt.datum["State Name"]
        == state_param  # Filter the chart based on the selected state
    )
    .properties(
        width=800,
        height=450,
        title=alt.TitleParams(
            "AQI Trends for Selected State (2000–2024)",
            font="Verdana",
            fontSize=18,
            anchor="start",
            color="#333333",  # Dark gray for a professional title color
        ),
    )
    .interactive()
)  # Make the chart interactive (zoom, pan)

# Trend line (linear regression line) with a different color
trend_line = (
    alt.Chart(state_yearly_aqi)
    .mark_line(color="red")
    .encode(
        x="Year:O",
        y="AQI:Q",
        tooltip=[
            alt.Tooltip("Year:O", title="Year"),
            alt.Tooltip("AQI:Q", title="AQI"),
        ],
    )
    .transform_filter(alt.datum["State Name"] == state_param)
    .transform_regression("Year", "AQI", method="linear")
)

# Combine the line chart and trend line
final_chart = (line_chart + trend_line).add_params(state_param)

# Display the final interactive chart with better styling
final_chart.configure_axis(
    grid=False,  # Turn off grid for a cleaner look
    labelFontSize=12,
    titleFontSize=14,
    labelFont="Verdana",
    titleFont="Verdana",
    labelColor="#333333",  # Dark gray for labels
    titleColor="#333333",  # Dark gray for titles
).configure_title(
    fontSize=20,
    font="Verdana",
    anchor="middle",
    color="#1f77b4",  # Title color to match the chart theme
).configure_legend(
    labelFontSize=12,
    titleFontSize=14,
    labelFont="Verdana",
    titleFont="Verdana",
    labelColor="#333333",
    titleColor="#333333",
).configure_view(
    strokeWidth=0  # Remove the border around the chart for a cleaner look
)

alt.LayerChart(...)